In [ ]:
!pip install opencv-python
!pip install tensorflow_hub


In [ ]:
import cv2
import tensorflow_hub as hub
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import os

In [ ]:
# Open the video file for reading
cam = cv2.VideoCapture("IMG_3508.MOV")

# Initialize a variable to keep track of the current frame number
currentframe = 0

# Start a loop to read frames from the video
while (True):
    # Read a frame from the video
    ret, frame = cam.read()
    
    # Check if the frame was read successfully
    if ret:
        # Define the name for the output image file
        name = f'frames/frame{currentframe}.jpg'
        
        # Save the frame as an image with the specified name
        cv2.imwrite(name, frame)
        
        # Increment the current frame number
        currentframe += 1
    else:
        # If the frame cannot be read, break out of the loop
        break

# Release the video capture object
cam.release()

# Close all OpenCV windows
cv2.destroyAllWindows()

In [ ]:
# Load a pre-trained styleGAN model from TensorFlow Hub
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# Define a function to load an image from a file and preprocess it
def load_image(img_path):
    img = tf.io.read_file(img_path)  # Read the image file
    img = tf.image.decode_image(img, channels=3)  # Decode the image with 3 color channels
    img = tf.image.convert_image_dtype(img, tf.float32)  # Convert image data type to float32
    img = img[tf.newaxis, :]  # Add a batch dimension to the image
    return img

# Load the style image using the load_image function
style_image = load_image('style.jpg')

# Get a list of image files in the 'frames' directory
frames_images = os.listdir('frames')

# Iterate through each frame image and apply stylization
for frame in range(len(frames_images)):
    # Load the current frame image using the load_image function
    content_image = load_image(f'frames/frame{frame}.jpg')
    
    # Apply stylization to the frame using the loaded model and style image
    stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]
    
    # Save the stylized frame as an image in the 'styled_frames' directory
    cv2.imwrite(f'styled_frames/generated_frame{frame}.jpg', cv2.cvtColor(np.squeeze(stylized_image)*255, cv2.COLOR_BGR2RGB))


In [ ]:
# Create an empty list to store the frames that will be used to generate the video
img_array = []

# Iterate through the generated stylized frames
for generated_frame in range(len(frames_images)):
    # Read a stylized frame image from the 'styled_frames' directory
    img = cv2.imread(f'styled_frames/generated_frame{generated_frame}.jpg')
    
    # Get the height, width, and number of color channels of the frame
    height, width, layers = img.shape
    
    # Define the size of the frame using its width and height
    size = (width, height)
    
    # Append the frame to the img_array list
    img_array.append(img)

# Create a VideoWriter object to write the frames into a video file
out = cv2.VideoWriter('styled_video.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

# Iterate through the list of frames and write them to the video
for i in range(len(img_array)):
    out.write(img_array[i])

# Release the VideoWriter, finalizing the video creation
out.release()